In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
print("Trainning data size:", mnist.train.num_examples)
print("Validating data size:", mnist.validation.num_examples)
print("Testing data size:", mnist.test.num_examples)

#print("Example trainning data:", mnist.train.images[0])
print("Example traingning data label:", mnist.train.labels[0])

print("Trainning data shape:", mnist.train.images.shape)

Trainning data size: 55000
Validating data size: 5000
Testing data size: 10000
Example traingning data label: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Trainning data shape: (55000, 784)


In [5]:
batch_size = 100
xs, ys = mnist.train.next_batch(batch_size)

print("X shape:", xs.shape)
print("Y shape:", ys.shape)

X shape: (100, 784)
Y shape: (100, 10)


In [30]:
INPUT_NODE = 784
OUTPUT_NODE = 10

LAYER1_NODE = 500
BATCH_SIZE = 100

LEARNING_RATE_BASE  = 0.8
LEARNING_RATE_DECAY = 0.99

REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99

In [31]:
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 当没有提供 '滑动平均类' 时, 直接使用参数当前的取值
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        layer2 = tf.matmul(layer1, weights2) + biases2
        return layer2
    else:
        # 首先使用 avg_class.average 来计算，得出变量的 '滑动平均值'
        # 然后在计算相应的神经网络向前传播结果
        mov_weights1 = avg_class.average(weights1)
        mov_biases1  = avg_class.average(biases1)
        mov_weights2 = avg_class.average(weights2)
        mov_biases2  = avg_class.average(biases2)
        layer1 = tf.nn.relu(tf.matmul(input_tensor, mov_weights1) + mov_biases1)
        layer2 = tf.matmul(layer1, mov_weights2) + mov_biases2
        return layer2

In [32]:
def train(mnist):
    x  = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    # 生成 Hidden-Layer 参数
    weight1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    weight2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    # 计算在当前参数下神经网络向前传播的结果
    # 这里给出的用于计算的 '滑动平均类' 为 None
    y = inference(x, None, weight1, biases1, weight2, biases2)
    
    # 定义存储训练轮数的变量
    # 这个变量不需要计算 '滑动平均类'，所以这里指定这个变量为不可训练的变量
    # 在使用 Tensorflow 训练神经网络时，一般会将代表训练轮数的变量指定为不可训练的变量
    global_step = tf.Variable(0, trainable=False)
    
    variable_averages =tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    
    # 在所有代表神经网络参数的变量上使用 '滑动平均类'
    # tf.trainable_variables 返回的就是图上集合 GraphKeys.TRAINABLE_VARIABLES 中的元素
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    # 计算使用了 '滑动平均类' 的向前传播结果
    average_y = inference(x, variable_averages, weight1, biases1, weight2, biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算 L2 正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weight1) + regularizer(weight2)
    
    loss = cross_entropy_mean + regularization
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step)
    
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op = tf.no_op(name='train')
    
    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
        test_feed = {x:mnist.test.images,y_:mnist.test.labels}
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _,loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            if i%1000==0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                validate_acc = sess.run(accuracy,feed_dict=validate_feed)
                print ("After %d training step(s),validation accuracy using average model is %g "%(step,validate_acc))
                test_acc = sess.run(accuracy,feed_dict=test_feed)
                print("After %d training step(s) testing accuracy using average model is %g"%(step,test_acc))

In [33]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
train(mnist)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
After 1 training step(s), loss on training batch is 2.86374.
After 1 training step(s),validation accuracy using average model is 0.0824 
After 1 training step(s) testing accuracy using average model is 0.0923
After 1001 training step(s), loss on training batch is 0.220504.
After 1001 training step(s),validation accuracy using average model is 0.9768 
After 1001 training step(s) testing accuracy using average model is 0.9754
After 2001 training step(s), loss on training batch is 0.163768.
After 2001 training step(s),validation accuracy using average model is 0.9802 
After 2001 training step(s) testing accuracy using average model is 0.9806
After 3001 training step(s), loss on training batch is 0.133581.
After 3001 training step(s),validation accuracy using average model is 0.9812 
After 3001 trai

KeyboardInterrupt: 